In [1]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("quora")

In [2]:
raw_df = dataset["train"].to_pandas()

In [3]:
raw_df.head()

,questions,is_duplicate
0,"{'id': [1, 2], 'text': ['What is the step by s...",False
1,"{'id': [3, 4], 'text': ['What is the story of ...",False
2,"{'id': [5, 6], 'text': ['How can I increase th...",False
3,"{'id': [7, 8], 'text': ['Why am I mentally ver...",False
4,"{'id': [9, 10], 'text': ['Which one dissolve i...",False


In [9]:
raw_df[['q1', 'id1', 'id2']].rename(columns={'q1': 'text', 'id1': 'id', 'id2': 'dq_id'})

,text,id,dq_id
0,What is the step by step guide to invest in sh...,1,2
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,3,4
2,How can I increase the speed of my internet co...,5,6
3,Why am I mentally very lonely? How can I solve...,7,8
4,"Which one dissolve in water quikly sugar, salt...",9,10
...,...,...,...
404285,How many keywords are there in the Racket prog...,433578,379845
404286,Do you believe there is life after death?,18840,155606
404287,What is one coin?,537928,537929
404288,What is the approx annual cost of living while...,537930,537931


In [8]:
raw_df[['id1', 'id2']] = pd.DataFrame(raw_df['questions'].apply(lambda x: x['id']).tolist(), index=raw_df.index)
raw_df[['q1', 'q2']] = pd.DataFrame(raw_df['questions'].apply(lambda x: x['text']).tolist(), index=raw_df.index)

# Preparing the flat DataFrame
flat_df = (
    raw_df[['q1', 'id1', 'id2']].rename(columns={'q1': 'text', 'id1': 'id', 'id2': 'dq_id'})
    .append(raw_df[['q2', 'id2', 'id1']].rename(columns={'q2': 'text', 'id2': 'id', 'id1': 'dq_id'}))
    .reset_index(drop=True)
)

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
flat_df

In [10]:
raw_df["q1"] = raw_df["questions"].apply(lambda q: q["text"][0])
raw_df["q2"] = raw_df["questions"].apply(lambda q: q["text"][1])
raw_df["id1"] = raw_df["questions"].apply(lambda q: q["id"][0])
raw_df["id2"] = raw_df["questions"].apply(lambda q: q["id"][1])

q1_to_q2 = raw_df.copy().rename(columns={"q1": "text", "id1": "id", "id2": "dq_id"}).drop(columns=["questions", "q2"])
q2_to_q1 = raw_df.copy().rename(columns={"q2": "text", "id2": "id", "id1": "dq_id"}).drop(columns=["questions", "q1"])
flat_df = pd.concat([q1_to_q2, q2_to_q1])

In [16]:
flat_df.loc[0]

,is_duplicate,id,dq_id,text
0,False,1,2,What is the step by step guide to invest in sh...
0,False,2,1,What is the step by step guide to invest in sh...




Next, we take a sample of the data and retaining just one row per question while storing a list of duplicated question IDs for each question.


In [12]:
df = flat_df.drop_duplicates("id").head(2000)
df["duplicated_questions"] = df["id"].apply(lambda qid: flat_df[flat_df["id"] == qid]["dq_id"].tolist())
df = df.drop(columns=["dq_id", "is_duplicate"])

df.tail()

,id,text,duplicated_questions
2003,3985,What name will you give to a mathematics educa...,"[3986, 535562]"
2004,3987,What universities does Federal-Mogul recruit n...,"[3988, 23638]"
2005,3989,Why do people go for mba after masters in engi...,"[3990, 161829, 316113, 168810]"
2006,3991,How does a facebook account get hacked?,"[3992, 379602, 35699]"
2007,3993,Why do black people get upset when non-black r...,[3994]


In [ ]:
## 이제 이걸 활용하여 classification을 할 거다

# embedding을 돌리고, 각 줄 별로 embedding 유사도를 구한다.
# 그래서 가져오는 것들이 원래 있던 것들과 같다면 pass
# 그렇지 않ㄴ다면 fail
